# Distances

In [1]:
from koselleck import *

## Generating data

### Across-model Distance matrices

In [4]:
# src(measure_dist_local)

In [5]:
# src(prep_dists_local_input)

In [6]:
# src(dists_local)

In [7]:
# # add to mongo
# veclib=get_veclib()
# with shelve.open(FN_VECLIB) as veclib1:
#     for k,v in tqdm(veclib1.items(), total=len(veclib1)):
#         if k.startswith('lnm(') and not 'k=' in k:
#             veclib[k]=v

In [ ]:
dfdist_long = dists_local(get_valid_words())
dfdist_long

Checking if cached: 100%|██████████| 6822144/6822144 [23:41<00:00, 4797.76it/s]
Already measured 6,822,144 local neighborhood distances


In [8]:
dfdist_long=dists_local('culture')
dfdist_long

Already measured 1,128 local neighborhood distances
Finished in 0.9 seconds


dist_local  \
corpus1 corpus2 period1   period2   word1   word2   qstr                                                           
bpo     bpo     1720-1725 1725-1730 culture culture lnm(culture_1720-1725_bpo,culture_1725-1730_bpo)    0.099142   
                          1730-1735 culture culture lnm(culture_1720-1725_bpo,culture_1730-1735_bpo)    0.087559   
                          1735-1740 culture culture lnm(culture_1720-1725_bpo,culture_1735-1740_bpo)    0.071810   
                          1740-1745 culture culture lnm(culture_1720-1725_bpo,culture_1740-1745_bpo)    0.068433   
                          1745-1750 culture culture lnm(culture_1720-1725_bpo,culture_1745-1750_bpo)    0.063414   
...                                                                                                          ...   
                1940-1945 1950-1955 culture culture lnm(culture_1940-1945_bpo,culture_1950-1955_bpo)    0.037797   
                          1955-1960 culture culture lnm(culture_1940-1945_bpo,culture_1955-1960_bpo)    0.073821   
                1945-1950 1950-1955 culture culture lnm(culture_1945-1950_bpo,culture_1950-1955_bpo)    0.030688   
                          1955-1960 culture culture lnm(culture_1945-1950_bpo,culture_1955-1960_bpo)    0.060311   
                1950-1955 1955-1960 culture culture lnm(culture_1950-1955_bpo,culture_1955-1960_bpo)    0.057241   

                                                                                                      metaneighb_size  \
corpus1 corpus2 period1   period2   word1   word2   qstr                                                                
bpo     bpo     1720-1725 1725-1730 culture culture lnm(culture_1720-1725_bpo,culture_1725-1730_bpo)           49.000   
                          1730-1735 culture culture lnm(culture_1720-1725_bpo,culture_1730-1735_bpo)           48.000   
                          1735-1740 culture culture lnm(culture_1720-1725_bpo,culture_1735-1740_bpo)           49.000   
                          1740-1745 culture culture lnm(culture_1720-1725_bpo,culture_1740-1745_bpo)           45.125   
                          1745-1750 culture culture lnm(culture_1720-1725_bpo,culture_1745-1750_bpo)           48.000   
...                                                                                                               ...   
                1940-1945 1950-1955 culture culture lnm(culture_1940-1945_bpo,culture_1950-1955_bpo)           47.000   
                          1955-1960 culture culture lnm(culture_1940-1945_bpo,culture_1955-1960_bpo)           49.000   
                1945-1950 1950-1955 culture culture lnm(culture_1945-1950_bpo,culture_1950-1955_bpo)           42.000   
                          1955-1960 culture culture lnm(culture_1945-1950_bpo,culture_1955-1960_bpo)           45.600   
                1950-1955 1955-1960 culture culture lnm(culture_1950-1955_bpo,culture_1955-1960_bpo)           47.000   

                                                                                                      neighb1_size  \
corpus1 corpus2 period1   period2   word1   word2   qstr                                                             
bpo     bpo     1720-1725 1725-1730 culture culture lnm(culture_1720-1725_bpo,culture_1725-1730_bpo)          25.0   
                          1730-1735 culture culture lnm(culture_1720-1725_bpo,culture_1730-1735_bpo)          25.0   
                          1735-1740 culture culture lnm(culture_1720-1725_bpo,culture_1735-1740_bpo)          25.0   
                          1740-1745 culture culture lnm(culture_1720-1725_bpo,culture_1740-1745_bpo)          25.0   
                          1745-1750 culture culture lnm(culture_1720-1725_bpo,culture_1745-1750_bpo)          25.0   
...                                                                                                            ...   
                1940-1945 1950-1955 culture culture lnm(culture_1940-1945_b

## Distance matrices

In [9]:
dfdist_wide=get_historical_semantic_distance_matrix('culture')
dfdist_wide

Already measured 1,128 local neighborhood distances
Finished in 0.9 seconds


period2,1720-1725,1725-1730,1730-1735,1735-1740,1740-1745,1745-1750,1750-1755,1755-1760,1760-1765,1765-1770,...,1910-1915,1915-1920,1920-1925,1925-1930,1930-1935,1935-1940,1940-1945,1945-1950,1950-1955,1955-1960
period1,,,,,,,,,,,,,,,,,,,,,
1720-1725,NaN,47.406915,40.403369,28.612589,26.218972,22.052305,41.023936,61.768617,52.371454,62.300532,...,65.137411,78.612589,70.899823,73.204787,70.101950,53.346631,52.992021,50.687057,43.683511,38.364362
1725-1730,47.406915,NaN,37.566489,29.942376,28.257979,26.484929,38.896277,59.109043,52.460106,57.158688,...,85.616135,91.023936,87.921099,91.999113,87.743794,81.183511,70.190603,68.860816,71.875000,54.765071
1730-1735,40.403369,37.566489,NaN,21.165780,18.328901,17.087766,38.541667,46.165780,32.335993,40.137411,...,69.658688,83.399823,76.218972,76.307624,74.179965,70.013298,58.488475,55.031028,59.375000,39.871454
1735-1740,28.612589,29.942376,21.165780,NaN,15.403369,13.364362,20.101950,20.988475,15.935284,30.296986,...,67.885638,83.843085,75.953014,77.637411,70.456560,64.339539,46.963652,44.570035,52.903369,43.594858
1740-1745,26.218972,28.257979,18.328901,15.403369,NaN,12.300532,28.346631,53.967199,44.658688,51.130319,...,59.729610,67.176418,57.867908,69.481383,56.626773,51.396277,44.038121,45.101950,46.609043,34.729610
1745-1750,22.052305,26.484929,17.087766,13.364362,12.300532,NaN,14.516844,34.995567,24.534574,38.984929,...,59.552305,67.619681,63.009752,68.506206,63.187057,53.435284,44.924645,39.428191,43.772163,31.006206
1750-1755,41.023936,38.896277,38.541667,20.101950,28.346631,14.516844,NaN,30.740248,23.648050,55.474291,...,75.243794,87.566489,84.109043,81.360816,74.445922,65.846631,50.509752,53.169326,57.247340,49.977837
1755-1760,61.768617,59.109043,46.165780,20.988475,53.967199,34.995567,30.740248,NaN,8.665780,33.399823,...,85.970745,93.417553,90.669326,93.240248,87.034574,77.371454,68.151596,59.640957,72.406915,66.112589
1760-1765,52.371454,52.460106,32.335993,15.935284,44.658688,24.534574,23.648050,8.665780,NaN,29.853723,...,73.382092,86.414007,80.385638,85.704787,78.169326,65.226064,52.814716,48.204787,62.743794,58.931738


In [10]:
@interact_manual
def show(w='culture'): return plot_historical_semantic_distance_matrix(w)

interactive(children=(Text(value='culture', description='w'), Button(description='Run Interact', style=ButtonS…